In [5]:
import os
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

data_list = []

csv_files = [f for f in os.listdir('data/')] 
for file in csv_files:
    file_path = os.path.join('data/', file)
    data_list.append(pd.read_csv(file_path))

df = pd.concat(data_list, ignore_index=True)

df[df['class'] == 'side-lean'] #Grabs only a specific class.

    

,class,x1,y1,z1,v1,x2,y2,z2,v2,x3,...,z32,v32,x33,y33,z33,v33,neck_tilt,shoulder_tilt,mouth_tilt,eye_tilt
2895,side-lean,0.492384,0.527363,-0.968108,0.999913,0.503271,0.458974,-0.916480,0.999859,0.516659,...,0.163879,0.004829,0.504237,3.173965,-0.088000,0.003047,25.267629,81.740799,76.627219,78.148955
2896,side-lean,0.492747,0.524276,-0.931260,0.999912,0.503737,0.458264,-0.878548,0.999858,0.517507,...,0.225460,0.004670,0.497704,3.164940,-0.081306,0.002999,25.385828,84.628972,77.826794,79.356813
2897,side-lean,0.493838,0.522626,-0.848553,0.999901,0.503768,0.457924,-0.797533,0.999843,0.517511,...,0.231065,0.004662,0.494831,3.116418,-0.109185,0.003050,25.288710,87.269056,76.974704,78.985861
2898,side-lean,0.495225,0.522564,-0.799285,0.999894,0.504001,0.457949,-0.747684,0.999833,0.517585,...,0.317378,0.004563,0.508939,3.064914,-0.091647,0.003069,25.267174,88.043783,75.974376,78.506766
2899,side-lean,0.497779,0.520518,-0.972503,0.999898,0.504337,0.456894,-0.916950,0.999838,0.517767,...,0.338285,0.004425,0.524355,3.029559,-0.078605,0.003038,25.082416,86.219471,76.347143,77.000172
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4187,side-lean,0.366662,0.637441,-1.458171,0.999642,0.367527,0.567462,-1.424804,0.999324,0.381046,...,0.265512,0.013852,0.425135,3.042473,0.052740,0.010990,8.477497,78.647139,49.535568,52.410159
4188,side-lean,0.371947,0.628478,-1.635954,0.999475,0.373340,0.558758,-1.606439,0.999207,0.386666,...,0.197854,0.014168,0.438295,3.021928,-0.059353,0.011245,10.848014,75.901968,53.909793,56.755713
4189,side-lean,0.377070,0.627853,-1.502358,0.999477,0.380223,0.558014,-1.468945,0.999209,0.393559,...,0.025165,0.014012,0.460928,2.984723,-0.176566,0.011123,13.036575,73.749378,55.937396,58.069421
4190,side-lean,0.384622,0.624425,-1.730209,0.999526,0.390024,0.554016,-1.685676,0.999283,0.402702,...,0.086636,0.012979,0.492708,2.988293,-0.145354,0.010147,14.245340,69.348280,60.528279,60.216504


In [6]:
X = df.drop('class', axis=1) #Features (no class col)
y = df['class'] #Target value (class col)


In [7]:
#Create train/ test split.
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1234)

# Train Classification Model

In [8]:
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression, RidgeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [16]:
pipelines = {
    'lr':make_pipeline(StandardScaler(), LogisticRegression(max_iter=200)),
    'rc':make_pipeline(StandardScaler(), RidgeClassifier(max_iter=200)),
    'rf':make_pipeline(StandardScaler(), RandomForestClassifier()),
    'gb':make_pipeline(StandardScaler(), GradientBoostingClassifier())
}

#Train every model (4) on the data.
fit_models = {}
for algo, pipeline in pipelines.items():
    model = pipeline.fit(X_train, y_train)
    fit_models[algo] = model

In [17]:
fit_models['lr'].predict(X_test)

array(['upright', 'upright', 'forward-slouch', ..., 'side-lean',
       'forward-slouch', 'forward-slouch'], dtype=object)

# Evaluate & Serialise Models

In [18]:
from sklearn.metrics import accuracy_score
import pickle

In [19]:
for algo, model in fit_models.items():
    yhat = model.predict(X_test)
    print(algo, accuracy_score(y_test, yhat))


lr 0.9908854166666666
rc 0.9759114583333334
rf 0.9915364583333334
gb 0.9915364583333334


In [26]:
# SAVE MODEL #
with open('pose_detection.pkl', 'wb') as f:
    pickle.dump(fit_models['lr'], f)

# Make Detections w/ Model

In [25]:
# RELOAD INTO NOTEBOOK #
with open('pose_detection.pkl', 'rb') as f:
    model = pickle.load(f)

